# Preprocessing

* 컬럼명은 다음과 같이 
    * comment, score
* 지울 것(Dataset에서 수행)
    * 개행
    * 쌍따옴표
    * 도메인주소, ip address
* 소문자화(tokenizer에서 수행)
* scoring
    * 레이블당 기준
        * 나머지는 그냥하면되고
        * severe_toxic을 어떻게 처리할 것인지
            * 직접 보고 판단 : toxic 보다 더 offensive한지, 맞다면 가중치를 더 주는 방향으로
            * servere_toxic이 더 offensive하다. 가중치를 부여한다.
                * 1st : servere_toxic만 2로
                * 2nd : servere_toxic과 toxic의 상관관계의 역수 (약 2.5)를 servere_toxic에 곱해서 사용한다.
    * method
        * sum : 단순 합은 부적절
        * avg
            * “발생 평균” : 선정
        * max
        * weighted sum/mean
    * ruddit data
        * [-1, 1] —> [0,1] 

## 1st Data

In [1]:
import pandas as pd
import numpy as np

In [26]:
# read data
train_1st = pd.read_csv('data/1st/train.csv')
# sample
train_1st.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [33]:
# severe_toxic은 가중치 부여 (1->2)
train_1st.loc[train_1st['severe_toxic']==1, 'severe_toxic'] = 2
train_1st['severe_toxic'].value_counts()

0    157976
2      1595
Name: severe_toxic, dtype: int64

In [36]:
# 발생 평균
train_1st['score'] = train_1st.iloc[:, 2:].replace(0, float('NaN')).mean(axis=1, skipna=True)
train_1st.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,score
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0,NaN
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0,NaN
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0,NaN
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0,NaN
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0,NaN


In [37]:
train_1st['score'] = train_1st['score'].fillna(0)

In [38]:
train_1st['score'].value_counts()

0.000000    143346
1.000000     14630
1.250000      1008
1.200000       329
1.333333       186
1.500000        41
1.166667        31
Name: score, dtype: int64

In [41]:
train_1st = train_1st[['comment_text', 'score']]
train_1st.columns = ['comment', 'score']
train_1st.head()

,comment,score
0,Explanation\nWhy the edits made under my usern...,0.0
1,D'aww! He matches this background colour I'm s...,0.0
2,"Hey man, I'm really not trying to edit war. It...",0.0
3,"""\nMore\nI can't make any real suggestions on ...",0.0
4,"You, sir, are my hero. Any chance you remember...",0.0


In [42]:
train_1st.to_csv('data/1st/train_1st.csv')

In [63]:
# read data
test = pd.read_csv('data/1st/test.csv')
test_labels = pd.read_csv('data/1st/test_labels.csv')
test.head()

,id,comment_text
0,00001cee341fdb12,Yo bitch Ja Rule is more succesful then you'll...
1,0000247867823ef7,== From RfC == \n\n The title is fine as it is...
2,00013b17ad220c46,""" \n\n == Sources == \n\n * Zawe Ashton on Lap..."
3,00017563c3f7919a,":If you have a look back at the source, the in..."
4,00017695ad8997eb,I don't anonymously edit articles at all.


In [64]:
test = pd.concat([test, test_labels.iloc[:, 1:]], axis=1)

In [65]:
test.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,00001cee341fdb12,Yo bitch Ja Rule is more succesful then you'll...,-1,-1,-1,-1,-1,-1
1,0000247867823ef7,== From RfC == \n\n The title is fine as it is...,-1,-1,-1,-1,-1,-1
2,00013b17ad220c46,""" \n\n == Sources == \n\n * Zawe Ashton on Lap...",-1,-1,-1,-1,-1,-1
3,00017563c3f7919a,":If you have a look back at the source, the in...",-1,-1,-1,-1,-1,-1
4,00017695ad8997eb,I don't anonymously edit articles at all.,-1,-1,-1,-1,-1,-1


In [62]:
test_labels['insult'].value_counts()

-1    89186
 0    60551
 1     3427
Name: insult, dtype: int64

In [66]:
len(test)

153164

In [67]:
# -1 인 데이터는 지움
test= test[test['toxic'] != -1]
len(test)

63978

In [68]:
# severe_toxic은 가중치 부여 (1->2)
test.loc[test['severe_toxic']==1, 'severe_toxic'] = 2
test.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
5,0001ea8717f6de06,Thank you for understanding. I think very high...,0,0,0,0,0,0
7,000247e83dcc1211,:Dear god this site is horrible.,0,0,0,0,0,0
11,0002f87b16116a7f,"""::: Somebody will invariably try to add Relig...",0,0,0,0,0,0
13,0003e1cccfd5a40a,""" \n\n It says it right there that it IS a typ...",0,0,0,0,0,0
14,00059ace3e3e9a53,""" \n\n == Before adding a new product to the l...",0,0,0,0,0,0


In [69]:
test.describe()

,toxic,severe_toxic,obscene,threat,insult,identity_hate
count,63978.000000,63978.000000,63978.000000,63978.000000,63978.000000,63978.000000
mean,0.095189,0.011473,0.057692,0.003298,0.053565,0.011129
std,0.293478,0.151043,0.233161,0.057334,0.225160,0.104905
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,1.000000,2.000000,1.000000,1.000000,1.000000,1.000000


In [70]:
# 발생 평균
test['score'] = test.iloc[:, 2:].replace(0, float('NaN')).mean(axis=1, skipna=True)
test['score'] = test['score'].fillna(0)
test.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,score
5,0001ea8717f6de06,Thank you for understanding. I think very high...,0,0,0,0,0,0,0.0
7,000247e83dcc1211,:Dear god this site is horrible.,0,0,0,0,0,0,0.0
11,0002f87b16116a7f,"""::: Somebody will invariably try to add Relig...",0,0,0,0,0,0,0.0
13,0003e1cccfd5a40a,""" \n\n It says it right there that it IS a typ...",0,0,0,0,0,0,0.0
14,00059ace3e3e9a53,""" \n\n == Before adding a new product to the l...",0,0,0,0,0,0,0.0


In [71]:
test['score'].value_counts()

0.000000    57735
1.000000     5876
1.250000      185
1.200000      140
1.333333       28
1.166667       14
Name: score, dtype: int64

In [72]:
test = test[['comment_text', 'score']]
test.columns = ['comment', 'score']
test.head()

,comment,score
5,Thank you for understanding. I think very high...,0.0
7,:Dear god this site is horrible.,0.0
11,"""::: Somebody will invariably try to add Relig...",0.0
13,""" \n\n It says it right there that it IS a typ...",0.0
14,""" \n\n == Before adding a new product to the l...",0.0


In [73]:
train_1st = pd.concat([train_1st, test])
print('len : ',len(train_1st))
train_1st.head()

len :  223549


,comment,score
0,Explanation\nWhy the edits made under my usern...,0.0
1,D'aww! He matches this background colour I'm s...,0.0
2,"Hey man, I'm really not trying to edit war. It...",0.0
3,"""\nMore\nI can't make any real suggestions on ...",0.0
4,"You, sir, are my hero. Any chance you remember...",0.0


In [74]:
train_1st.to_csv('data/1st/train_1st.csv', index=False)

## Ruddit

In [92]:
ruddit_path = 'data/ruddit/ruddit_with_text_1.csv'
ruddit_data = pd.read_csv(ruddit_path)
ruddit_data.head()

,post_id,comment_id,txt,url,offensiveness_score
0,42g75o,cza1q49,> The difference in average earnings between m...,https://www.reddit.com/r/changemyview/comments...,-0.083
1,42g75o,cza1wdh,"The myth is that the ""gap"" is entirely based o...",https://www.reddit.com/r/changemyview/comments...,-0.022
2,42g75o,cza23qx,[deleted],https://www.reddit.com/r/changemyview/comments...,0.167
3,42g75o,cza2bw8,The assertion is that women get paid less for ...,https://www.reddit.com/r/changemyview/comments...,-0.146
4,42g75o,cza2iji,You said in the OP that's not what they're mea...,https://www.reddit.com/r/changemyview/comments...,-0.083


In [93]:
# select columns and rename columns
ruddit_data = ruddit_data[['txt', 'offensiveness_score']]
ruddit_data.columns = ['comment', 'score']

In [94]:
# remove '[deleted]' rows
ruddit_data = ruddit_data[ruddit_data['comment'] != '[deleted]']
len(ruddit_data)

453

In [95]:
# min max scaling (0~1 범위로 변환)
from sklearn.preprocessing import MinMaxScaler
min_max_scaler = MinMaxScaler()

In [96]:
ruddit_data['score'] = min_max_scaler.fit_transform(ruddit_data['score'].to_numpy().reshape(-1,1)).reshape(-1)
ruddit_data.describe()

,score
count,453.000000
mean,0.477985
std,0.171724
min,0.000000
25%,0.375457
50%,0.482104
75%,0.588751
max,1.000000


In [97]:
ruddit_data.head()

,comment,score
0,> The difference in average earnings between m...,0.436815
1,"The myth is that the ""gap"" is entirely based o...",0.481373
3,The assertion is that women get paid less for ...,0.390796
4,You said in the OP that's not what they're mea...,0.436815
5,>Men and women are not payed less for the same...,0.466764


In [98]:
ruddit_data.to_csv('data/ruddit/ruddit.csv', index=False)